In [1]:
import logging
import os
import pandas as pd
import src.constants as ct


from src.parsedata import PDFParser
from src.initialize import Logger
from src.capitalgains import CapitalGains
from src.utils import rename_mf_scheme


pd.options.display.max_columns = 100
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
!python --version

Python 3.9.5


In [2]:
logger = Logger()
logger.initilaze_logging('no')   
logging.info(f'{"*" * 50}Start Of Process{"*" * 50}')




## Pre-Requisite: CAS statement
The first step is to get the [consolidated account statement](https://www.camsonline.com/Investors/Statements/Consolidated-Account-Statement) from CAMS in PDF Format. Make sure that you select the option as highlighted in yellow.

 <!-- ![screenshot](https://github.com/ajayrawatsap/mf_data/blob/master/data/assets/cams.PNG) -->

 <img src="data/assets/cams.PNG" width=800 height=400 />




## Validate if CAS file exists

Rename your CAS statement from CAMS to *cas.pdf* and place it at root directory. It should be in same directory as this notebook demo.ipynb.
This applies to both local as well as cloud VM enviornment.

In [3]:
import os.path
file_name = 'cas.pdf'
if not os.path.isfile(file_name):
    raise ValueError("file cas.pdf not found in root folder: Hint: Rename your CAS statement file to cas.pdf and save in root directory ")
else: 
    print(f'File {file_name} Found: OK')

File cas.pdf Found: OK


##  Password Input

The password is same as what you set up when requesting the statement from CAMS website.

In [4]:
from getpass import getpass
password = getpass(prompt='Input your password for cas.pdf: ')


## Read Contents of PDF

In [5]:
%%time
print('Parsing CAS statement Data Please wait:', end = " ")
dp = PDFParser()    
mf_trans_df, mf_hdr_df = dp.parse_mf_data(file_name, password)
print("Done")

# mf_hdr_df = mf_hdr_df.loc[[0,2,7,9,15]]

Parsing CAS statement Data Please wait: Done
Wall time: 621 ms


## Calculate LTCG and STCG
By defaut Target LTCG amount is set to INR 100,000. IF you want to set it to different value (e.g.. INR 50,000) change the code in cell below and then run all cells again

<code>target_ltcg = 50000</code>

In [6]:
print('Calculating LTCG and Units to Sell: ', end = " ")

target_ltcg = 100000

cg = CapitalGains(mf_hdr_df, mf_trans_df)
cg.prepare_final_data(target_ltcg)
print("Done")

Calculating LTCG and Units to Sell:  Done


## Display Header Data (With units to Sell)

<BR>The Header and Transaction Data gets saved in data/output directory of Cloud VM or Local Machine as csv files. For Cloud VM Select the checkbox againt each file and download it to your desktops

In [7]:
from src.mfdisplay import get_hdr_data_to_display

hdr_df = get_hdr_data_to_display(cg.output_hdr_df)
hdr_df

,scheme_name,type,latest_nav,latest_nav_date,gf_nav,invested_amt,units,ltcg,stcg,current_amt,perc_gain,target_ltcg,target_units,target_amt,comments
0,DSP Small Cap Fund - Direct Plan - Growth,EQUITY,108.83,05-Aug-2021,70.16,"₹759,989.98","13,196.65","₹407,157.01","₹90,142.79","₹1,436,151.72",88.97,"₹100,000.00","2,586.38","₹281,468.55","Sell 2,586.385 units or ₹ 281,468.55 for LTCG of 100,000.00"
1,Franklin India Flexi Cap Fund - Direct Plan - Growth (erstwhile Franklin India Equity Fund - Direct),EQUITY,950.19,05-Aug-2021,637.61,"₹920,001.44","1,667.94","₹561,259.55",₹0.00,"₹1,584,854.58",72.27,"₹100,000.00",319.92,"₹303,988.12","Sell 319.925 units or ₹ 303,988.12 for LTCG of 100,000.00"
2,Franklin India Low Duration Fund - Direct Plan - Growth,DEBT,24.42,05-Aug-2021,19.98,"₹33,068.34","1,758.42","₹9,876.88",₹0.00,"₹42,945.21",29.87,"₹9,876.88","1,758.42","₹42,945.21","Total LTCG 9,876.87 is less than target LTCG 100,000.00: Sell 1,758.421 units or ₹42,945.21 for LTCG of 9,876.87"
3,Franklin India Smaller Companies Fund - Direct Plan - Growth,EQUITY,91.74,05-Aug-2021,65.59,"₹696,999.96","12,301.35","₹408,253.89",₹0.00,"₹1,128,492.84",61.91,"₹100,000.00","3,824.49","₹350,848.08","Sell 3,824.487 units or ₹ 350,848.08 for LTCG of 100,000.00"
4,Franklin India Ultra Short Bond Fund Super Institutional Plan - Direct Plan - Growth,DEBT,30.90,05-Aug-2021,23.80,"₹33,847.65","1,653.00","₹17,238.15",₹0.00,"₹51,085.80",50.93,"₹17,238.15","1,653.00","₹51,085.80","Total LTCG 17,238.15 is less than target LTCG 100,000.00: Sell 1,653.000 units or ₹51,085.80 for LTCG of 17,238.15"
5,HDFC Top 100 Fund - Direct Plan - Growth Option,EQUITY,681.15,05-Aug-2021,498.03,"₹569,833.21","1,331.04","₹245,964.12",₹0.00,"₹906,634.61",59.10,"₹100,000.00",546.10,"₹371,973.11","Sell 546.099 units or ₹ 371,973.11 for LTCG of 100,000.00"
6,ICICI Prudential All Seasons Bond Fund - Direct Plan - Growth,DEBT,29.94,05-Aug-2021,21.84,"₹699,980.02","29,838.99","₹189,045.40","₹4,368.89","₹893,394.31",27.63,"₹100,000.00","8,640.20","₹258,692.05","Sell 8,640.205 units or ₹ 258,692.05 for LTCG of 100,000.00"
7,ICICI Prudential Banking and PSU Debt Fund - Direct Plan - Growth,DEBT,26.19,05-Aug-2021,19.87,"₹799,975.00","36,754.76","₹156,965.16","₹5,802.99","₹962,743.15",20.35,"₹100,000.00","11,114.32","₹291,125.21","Sell 11,114.322 units or ₹ 291,125.21 for LTCG of 100,000.00"
8,ICICI Prudential Long Term Equity Fund (Tax Saving) - Direct Plan - Growth,EQUITY,601.43,05-Aug-2021,382.19,"₹150,000.23",379.87,"₹78,466.79",₹0.00,"₹228,467.01",52.31,"₹78,466.79",379.87,"₹228,467.02","Total LTCG 78,466.79 is less than target LTCG 100,000.00: Sell 379.873 units or ₹228,467.02 for LTCG of 78,466.79"
9,ICICI Prudential Nifty Next 50 Index Fund - Direct Plan - Growth,EQUITY,36.20,05-Aug-2021,27.51,"₹277,741.26","11,583.47","₹141,606.87",₹0.00,"₹419,348.14",50.98,"₹100,000.00","8,797.17","₹318,477.79","Sell 8,797.170 units or ₹ 318,477.79 for LTCG of 100,000.00"


### Display Target Units amd Amount to Redeem

In [8]:
   
from src.mfdisplay import display_target_units_amt
display_target_units_amt(cg.mf_hdr_df)

## Display Transaction Data for Single MF scheme

By default this will show transaction for a random Scheme. You can execute the cell multple times to see results for a diffrent scheme

<br>To view transaction for a specific scheme (e.g. DSP Small Cap Fund - Direct Plan - Growth)  replace  the code in cell as:-

<br> replace <code>scheme_name =  None</code> to
<br> <code> scheme_name = 'DSP Small Cap Fund - Direct Plan - Growth' </code>

Place cursor on code cell below Hit SHIFT + ENTER to see the results 


#### The *cumil_ltcg* and *cumil_units* columns are useful to know what units to sell for target ltcg.   Sell *cumil_units* for a LTCG of *cumil_ltcg*

In [9]:
from src.mfdisplay import get_trans_data_to_display, get_trans_for_single_scheme
trans_df = get_trans_data_to_display(cg.output_trans_df)

scheme_name =  None

get_trans_for_single_scheme(trans_df, scheme_name= scheme_name)


,scheme_name,trans_date,amount,units,purch_nav,trans_type,type,latest_nav,latest_nav_date,gf_nav,units_redeemed,units_remain,invested_amt,current_amt,perc_gain,new_purch_nav,ltcg,stcg,cumil_ltcg,cumil_units
239,Mirae Asset Large Cap Fund - Direct Plan - Growth,19-Feb-2019,250000.00,"5,098.61",49.03,PURCHASE,EQUITY,80.34,05-Aug-2021,51.80,0,5098.607,"₹250,000.00","₹409,637.38",63.85,49.03,"₹159,637.39",₹0.00,"₹159,637.39","5,098.61"
240,Mirae Asset Large Cap Fund - Direct Plan - Growth,10-Jun-2019,150000.00,"2,708.12",55.39,PURCHASE,EQUITY,80.34,05-Aug-2021,51.80,0,2708.119,"₹150,000.00","₹217,578.40",45.05,55.39,"₹67,578.40",₹0.00,"₹227,215.79","7,806.73"
241,Mirae Asset Large Cap Fund - Direct Plan - Growth,14-Jun-2019,300000.00,"5,464.68",54.90,PURCHASE,EQUITY,80.34,05-Aug-2021,51.80,0,5464.680,"₹300,000.00","₹439,048.79",46.35,54.90,"₹139,048.78",₹0.00,"₹366,264.57","13,271.41"
242,Mirae Asset Large Cap Fund - Direct Plan - Growth,17-Mar-2020,200000.00,"4,548.45",43.97,PURCHASE,EQUITY,80.34,05-Aug-2021,51.80,0,4548.452,"₹199,999.98","₹365,436.28",82.72,43.97,"₹165,436.30",₹0.00,"₹531,700.86","17,819.86"


## Portfolio Summary

In [10]:
from src.mfdisplay import group_by_mf_type, display_summary
mf_totals_grp = group_by_mf_type(cg.mf_hdr_df)
display_summary(mf_totals_grp)



            

##  Equiy vs Debt Aggregate Data

In [11]:
from src.mfdisplay import display_equity_vs_debt
display_equity_vs_debt(mf_totals_grp)

## Plot Mutual Funds Scheme Amounts

In [12]:
from src.mfdisplay import display_mf_scheme_amounts


display_mf_scheme_amounts(cg.mf_hdr_df)                 

## Plot Unrealised LTCG and STCG

In [13]:
from src.mfdisplay import display_unrealised_ltcg_stcg
display_unrealised_ltcg_stcg(cg.mf_hdr_df)               

## Plot Mutual Funds Allocation by Scheme

In [14]:
from src.mfdisplay import display_mf_scheme_pie_chart

display_mf_scheme_pie_chart(cg.mf_hdr_df)

In [15]:

logging.info(f'{"*" * 50}End Of Process{"*" * 50}\n') 

In [16]:
import requests

from datetime import datetime

def get_historical_nav(scheme_code:str)->pd.DataFrame:
    URL = 'https://api.mfapi.in/mf/' + scheme_code
    r = requests.get(URL)
    nav = r.json()

    nav_df = pd.json_normalize(nav['data'])
    nav_df['date'] = pd.to_datetime( nav_df['date'], format = '%d-%m-%Y' ).dt.date

    return nav_df


def get_portfolio_history_for_scheme(scheme_code:str, mf_trans_df:pd.DataFrame)->pd.DataFrame:

    '''
    For every day between first date of MF Transaction and Current date Calculate the Total Invested 
    Amount and Current Portfolio Aoount
    '''

    # Get relevant trasnaction data for a scheme
    cols = ['scheme_name', 'trans_date', 'invested_amt', 'units','units_remain', 'purch_nav', 'scheme_code']    
    # scheme_code = cg.mf_hdr_df[cg.mf_hdr_df.scheme_name == scheme_name]['scheme_code'].tolist()[0]
    df_mf = cg.mf_trans_df[cg.mf_trans_df.scheme_code == scheme_code][cols]

    
    df  = pd.DataFrame()
    #first data of MF transaction
    sdate = df_mf.iloc[0].trans_date
    # End Data Today
    edate = datetime.now().date()


    # Create a time series of dates 
    df['date'] = pd.date_range(sdate,edate,freq='d')
    df['date'] = df['date'].dt.date

    # Set same Scheme Name for all dates
    df['scheme_name'] = df_mf.iloc[0].scheme_name


    #Merge Time series with MF transaction data
    df = pd.merge(df, df_mf, how = 'left', left_on  =  ['scheme_name', 'date'], right_on  =  ['scheme_name', 'trans_date'] )    

    
   
    # Read Historical NAV for scheme from HTTP API
    nav_df =  get_historical_nav(scheme_code)

    #Set NAV for all dates 
    df = pd.merge(df, nav_df, on = 'date', how = 'left')

    #Replace missing NAV values from preceding valid value. 
    # NAV for Sat and Sun is same as Friday
    df['nav'] = df['nav'].fillna(method='ffill')

    #Set NULL values to 0 for numeric columns 
    num_cols = ['invested_amt', 'units', 'units_remain', 'purch_nav', 'nav']
    df[num_cols] = df[num_cols].fillna(0)
    df[num_cols] = df[num_cols].astype('float')


    # Total Unit Balance on each day
    # df['units_bal'] = df['units_remain'].cumsum()
    df['units_bal'] = df['units'].cumsum()


    #Total Current Amount  and Invested Amount at each date
    df['current_amt'] =  round(df['units_bal'] *  df['nav'], 2)
    df['invested_amt'] = df['invested_amt'].cumsum()

    return df



In [17]:
# for scheme_code in cg.mf_hdr_df.scheme_code.tolist():
#     mf_history = get_portfolio_history_for_scheme(scheme_code, cg.mf_trans_df)
#     mf_history.tail()

#     df = mf_history
#     scheme_name = df.iloc[0].scheme_name

#     from plotly import graph_objects as go

#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=df.date, y=df.invested_amt,
#                         mode='lines',
#                         name='Invested Amount'))

#     fig.add_trace(go.Scatter(x=df.date, y=df.current_amt,
#                         mode='lines',
#                         name='Current Amount'))
#     fig.update_layout(title = scheme_name )              
#     fig.show()

In [18]:


scheme_code = '118834'
mf_history = get_portfolio_history_for_scheme(scheme_code, cg.mf_trans_df)
mf_history.tail()

df = mf_history
scheme_name = df.iloc[0].scheme_name

from plotly import graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.date, y=df.invested_amt,
                    mode='lines',
                    name='Invested Amount'))

fig.add_trace(go.Scatter(x=df.date, y=df.current_amt,
                    mode='lines',
                    name='Current Amount'))
fig.update_layout(title = scheme_name )              
fig.show()

In [19]:
def get_trans_disp(df, scheme_name):
    del_cols = ['units_balance', 'amc',	'folio_no',	'type',	'isin',	'scheme_code',	'latest_nav_date', 'gf_nav', 'perc_gain',	'new_purch_nav']  
    disp_cols = [col for col in df.columns.tolist() if col not in del_cols]
    df = df[df.scheme_name == scheme_name]
    return df[disp_cols]

# scheme_name = cg.mf_trans_df.iloc[0].scheme_name    
get_trans_disp(cg.mf_trans_df, scheme_name)

,scheme_name,trans_date,amount,units,purch_nav,trans_type,latest_nav,units_redeemed,units_remain,invested_amt,current_amt,ltcg,stcg,cumil_ltcg,cumil_units
210,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-03-22,25000.00,451.353,55.389,PURCHASE_SIP,100.240,0,451.353,24999.99,45243.62,20243.633403,0,20243.633403,451.353
211,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-04-22,25000.00,446.859,55.946,PURCHASE_SIP,100.240,0,446.859,24999.97,44793.15,19793.172546,0,40036.805949,898.212
212,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-05-21,25000.00,450.013,55.554,PURCHASE_SIP,100.240,0,450.013,25000.02,45109.30,20109.280918,0,60146.086867,1348.225
213,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-06-21,25000.00,439.090,56.936,PURCHASE_SIP,100.240,0,439.090,25000.03,44014.38,19014.353360,0,79160.440227,1787.315
214,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-07-22,25000.00,452.096,55.298,PURCHASE_SIP,100.240,0,452.096,25000.00,45318.10,20318.098432,0,99478.538659,2239.411
215,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-08-21,25000.00,464.037,53.875,PURCHASE_SIP,100.240,0,464.037,24999.99,46515.07,21515.075505,0,120993.614164,2703.448
216,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-09-23,25000.00,432.331,57.826,PURCHASE_SIP,100.240,0,432.331,24999.97,43336.86,18336.887034,0,139330.501198,3135.779
217,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-10-22,25000.00,433.975,57.607,PURCHASE_SIP,100.240,0,433.975,25000.00,43501.65,18501.656175,0,157832.157373,3569.754
218,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-11-21,25000.00,416.354,60.045,PURCHASE_SIP,100.240,0,416.354,24999.98,41735.32,16735.349030,0,174567.506403,3986.108
219,Mirae Asset Emerging Bluechip Fund - Direct Plan - Growth,2019-12-23,25000.00,409.614,61.033,PURCHASE_SIP,100.240,0,409.614,24999.97,41059.71,16059.736098,0,190627.242501,4395.722


In [20]:
cg.mf_hdr_df[['scheme_name', 'scheme_code']]

,scheme_name,scheme_code
0,DSP Small Cap Fund - Direct Plan - Growth,119212
1,Franklin India Flexi Cap Fund - Direct Plan - Growth (erstwhile Franklin India Equity Fund - Direct),118535
2,Franklin India Low Duration Fund - Direct Plan - Growth,118530
3,Franklin India Smaller Companies Fund - Direct Plan - Growth,118525
4,Franklin India Ultra Short Bond Fund Super Institutional Plan - Direct Plan - Growth,118560
5,HDFC Top 100 Fund - Direct Plan - Growth Option,119018
6,ICICI Prudential All Seasons Bond Fund - Direct Plan - Growth,120603
7,ICICI Prudential Banking and PSU Debt Fund - Direct Plan - Growth,120256
8,ICICI Prudential Long Term Equity Fund (Tax Saving) - Direct Plan - Growth,120592
9,ICICI Prudential Nifty Next 50 Index Fund - Direct Plan - Growth,120684


In [21]:
history_df = pd.DataFrame()
# for scheme_code in ['119212', '118825']:
for scheme_code in cg.mf_hdr_df.scheme_code.tolist():
    df  = get_portfolio_history_for_scheme(scheme_code, cg.mf_trans_df)
    history_df  =  pd.concat([history_df, df])   

history_grp_df = history_df.groupby('date').agg({'current_amt':'sum'})

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=history_grp_df.index, y=history_grp_df.current_amt,
                    mode='lines',
                    name='Invested Amount'))

fig.show()

In [23]:
history_grp_df

,current_amt
date,
2015-10-23,9999.88
2015-10-24,9999.88
2015-10-25,9999.88
2015-10-26,9951.54
2015-10-27,9977.67
...,...
2022-01-04,14252183.71
2022-01-05,14272271.50
2022-01-06,14218936.93
